In [ ]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import (
    PlayerGameLog,
    BoxScorePlayerTrackV2,
    BoxScoreAdvancedV2,
    DraftCombinePlayerAnthro,
    CommonPlayerInfo
)
import pandas as pd


pid = players.find_players_by_full_name("Jayson Tatum")[0]["id"]


logs = PlayerGameLog(
    player_id=pid,
    season="2023-24",
    season_type_all_star="Playoffs"
).get_data_frames()[0]


finals_logs = logs[logs["MATCHUP"].str.contains("DAL")]
assert not finals_logs.empty, "No Finals logs found — check filters."


game_id = finals_logs.iloc[0]["Game_ID"]
print(f"Using Finals Game_ID: {game_id}")


trk = BoxScorePlayerTrackV2(game_id=game_id).player_track.get_data_frame()
spd_data = trk[trk["PLAYER_ID"] == pid][["PLAYER_NAME", "SPD", "DIST", "TCHS", "PASS"]]


adv = BoxScoreAdvancedV2(game_id=game_id).player_stats.get_data_frame()
adv_data = adv[adv["PLAYER_ID"] == pid][["PLAYER_NAME", "MIN", "PACE", "USG_PCT", "TS_PCT"]]


anthro = DraftCombinePlayerAnthro().get_data_frames()[0]
anthro_data = anthro[anthro["PLAYER_ID"] == pid][
    ["PLAYER_NAME", "HEIGHT_WO_SHOES", "HEIGHT_W_SHOES", "WEIGHT", "WINGSPAN"]
]

if anthro_data.empty:
    info = CommonPlayerInfo(player_id=pid).common_player_info.get_data_frame().iloc[0]
    anthro_data = pd.DataFrame([{
        "PLAYER_NAME": info["DISPLAY_FIRST_LAST"],
        "HEIGHT_W_SHOES": info["HEIGHT"],
        "WEIGHT": info["WEIGHT"],
        "WINGSPAN": None
    }])


print("\n--- Finals Game Data ---")
print(spd_data.to_string(index=False))
print(adv_data.to_string(index=False))
print("\n--- Player Anthropometrics ---")
print(anthro_data.to_string(index=False))


Using Finals Game_ID: 0042300405


JSONDecodeError: Expecting value: line 1 column 1 (char 0)